# Open Access database

In [28]:
import pyodbc
drvs = [x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]
print(len(drvs))
if len(drvs):
    for drvs1 in drvs:
        print(drvs1)

1
Microsoft Access Driver (*.mdb, *.accdb)


In [29]:
dbfile = r'path2file;'

con_str = r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)}; ' + f'DBQ={dbfile};'
con = pyodbc.connect(con_str)
cur = con.cursor()

In [30]:
tables = [table1 for table1 in cur.tables(tableType='TABLE')]
for table1 in tables:
    print(table1.table_name)

___AMAEM
___CAN
___CGU
___CR MONFORTE
___SCHA
___usuarios_AMAEM
___usuarios_CAN
___usuarios_CGU
___usuarios_MD
___usuarios_MONFORTE
___usuarios_SCHA
_Balsas usuarios JCUAV
_Bernedo_Basicos
_Bernedo_Explotaciones
_CAPT_DENTRO_SE_VINALOPO_DEF_11_07_5_2
_CAPTADORES
_contadores dudosos
_Copia seguridad de cruces_maria
_COPSEG_POZOS
_Datos Otros estudios recientes
_DATOS_CALIDAD_02/03/2011
_DATOS_CALIDAD_09/03/2011
_Datos_explo_recientes_otros estudios
_Datos_niveles_recientes_Otros estudios
_edu_calidad
_EDU_RESUMEN_EXTRACCIONES20110520
_eerr_av1
_Equival_cod2006_2009
_InscripIdentifEnCampoConConta0
_javier_conduc_usu
_javier_costes_energia
_javier_pozos_UDA
_JL_Comparativa_Dotaciones
_JL_Cultivos_Titular_Udas_Maria
_JL_porcentajes_Baterias JCUAV
_MAria_Asig Masub
_Maria_CostePond
_Maria_DAtosCostesEncuestados
_Maria_InscriConPozosEnDistintasMAsub
_Maria_Inscrip_excel
_Maria_Inscrip_ViToma
_Maria_Uda_AreaRiego
_Maria_vol por entidad_JC
_MariaAbto2
_MariaBateriasSinUDa
_MariaCostePNPEduResu

In [44]:
table1 = 'mytable'
for row in cur.columns(table=table1):
    print(row.column_name, row.type_name)

Codigo VARCHAR
fecha DATETIME
situaci VARCHAR
pnp DOUBLE


## Exportar db to csv

### Function

In [57]:
import csv
from os.path import join

def to_csv(cur, table1, direc, schema, w_csv=True, w_sql=True, encoding='UTF-8'):
    """
    
    """
    a2p_types = {'VARCHAR': 'varchar', 'INTEGER': 'integer', 'REAL': 'float32', 'DOUBLE': 'float32',
            'DATETIME': 'timesatmp'}

    column_names = [row.column_name.lower().strip().replace(' ', '_') for row in cur.columns(table=f'{table1}')]
    column_types = [row.type_name for row in cur.columns(table=f'{table1}')]

    if w_csv:
        cur.execute(f'select * from {table1}')

        csv_file = join(direc, table1.lower())
        with open(f'{csv_file}.csv', 'w', newline='') as fo:
            wfo = csv.writer(fo)
            wfo.writerow(column_names)
            for row in cur.fetchall():
                wfo.writerow(row)

    if w_sql:
        sql_file = join(direc, f'{table1.strip().lower()}') + '.sql'
        with open(f'{sql_file}', 'w', newline='') as fo:
            table1 = f'{schema}.{table1.strip().lower()}'
            fo.write(f'create table if not exists {table1}(\n')
            for i, (col_name, col_type) in enumerate(zip(column_names, column_types)):
                fo.write(f'{col_name} {a2p_types[col_type]}')
                if i < len(column_names) - 1:
                    fo.write(',\n')

            pk_cols = [row[8] for row in cur.statistics(table1) if row[5]=='PrimaryKey']
            if len(pk_cols) > 0:
                pk_cols = ', '.join(pk_cols)
                fo.write(',\n')
                fo.write(f'{pk_cols}\n);')
            else:
                fo.write('\n);')
        
            str_col_names = ', '.join(column_names)
            csv_file = join(direc, table1.lower())
            stm = f"copy {table1} ({str_col_names}) from '{csv_file}' with CSV header delimiter ',' encoding '{encoding}';"
            fo.write('\n\n')
            fo.write(f'{stm}\n')

In [58]:

direc = r'path2' 
table1 = 'mytable'
schema = 'myschema'

to_csv(cur, table1, direc, schema)

print('end')

terminado
